In [20]:
## Linear regression
# 1. Data definition
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [21]:
# 2. Hypothesis
# 학습 데이터와 잘 맞는 직선을 찾자
# y=Wx + b(weight+bias)

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
W = torch.zeros(1, requires_grad=True) # 이 두개 변수를 학습시키고 싶다.
b = torch.zeros(1, requires_grad=True) # requires_grad = True
hypothesis = x_train*W + b

In [22]:
# 3. compute loss
# 우리의 모델이 얼마나 정답과 가까운가 == cost, loss
# Mean Squared Error(MSE)
cost = torch.mean((hypothesis - y_train) ** 2)

In [23]:
# 4. Gradient descent
optimizer = optim.SGD([W, b], lr=0.01) # 학습할 tensors, learning rate

nb_epochs = 1000
for epoch in range(1, nb_epochs+1):
    hypothesis = x_train * W + b
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    optimizer.zero_grad() # gradient 초기화
    cost.backward() # gradient 계산
    optimizer.step() # 개선
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch  100/1000 W: 1.745, b: 0.579 Cost: 0.048403
Epoch  200/1000 W: 1.800, b: 0.456 Cost: 0.029910
Epoch  300/1000 W: 1.842, b: 0.358 Cost: 0.018483
Epoch  400/1000 W: 1.876, b: 0.281 Cost: 0.011421
Epoch  500/1000 W: 1.903, b: 0.221 Cost: 0.007058
Epoch  600/1000 W: 1.923, b: 0.174 Cost: 0.004361
Epoch  700/1000 W: 1.940, b: 0.137 Cost: 0.002695
Epoch  800/1000 W: 1.953, b: 0.107 Cost: 0.001665
Epoch  900/1000 W: 1.963, b: 0.084 Cost: 0.001029
Epoch 1000/1000 W: 1.971, b: 0.066 Cost: 0.000636


In [9]:
## High-level Implementation with nn.Module
# 이제 linear regression model을 만들게 되는데, 기본적으로 PyTorch의 모든 모델은 제공되는 nn.Module을 inherit해서 만들게된다.
class LinearRegressionModel(nn.Module):
    def __init__(self): # 사용할 layer들을 정의, 우리는 linear regression model을 만들 것이므로 nn.linear를 이용.
        super().__init__()
        self.linear = nn.Linear(1,1)
    def forward(self, x): # 이 모델이 어떻게 입력값에서 출력값을 계산하는지 알려줌
        return self.linear(x)

In [10]:
model = LinearRegressionModel()

In [11]:
## Hypothesis
# 이제 모델을 생성해서 예측값 H(x)를 구하자.

hypothesis = model(x_train)
print(hypothesis)

tensor([[0.1650],
        [0.3112],
        [0.4575]], grad_fn=<AddmmBackward>)


In [12]:
## Cost
# 이제 MSE로 cost를 구하자. MSE 역시 PyTorch 에서 기본적으로 제공한다.
print(hypothesis)
print(y_train)

tensor([[0.1650],
        [0.3112],
        [0.4575]], grad_fn=<AddmmBackward>)
tensor([[2.],
        [4.],
        [6.]])


In [13]:
cost = F.mse_loss(hypothesis, y_train)

In [14]:
print(cost)

tensor(15.8978, grad_fn=<MseLossBackward>)


In [16]:
## Gradient Descent
# 마지막 주어진 cost를 이용해 H(x)의 W, b를 바꾸어서 cost를 줄여준다. 이때 PyTorch의 torch.optim에 있는 optimizer들 중 하나를 사용.
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [17]:
optimizer.zero_grad()
cost.backward()
optimizer.step()

In [24]:
## Training with Full Code
# 이제 linear Regression 코드를 이해햇으니, 실제로 코드를 돌려 피팅시켜보자.

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
model = LinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 역전파 단계 전에, 갱신할 변수들에 ㅐ한 모든 변화도를 0으로 만들어야함. 
    # 기본적으로 .backward()를 할 때마다 변화도가 buffer에 누적되기 때문이다. torch.autograd.backward 문서 참조.
    cost.backward() # 모델의 매개변수에 대한 손실의 변화도를 개선
    optimizer.step() # 매개변수가 갱신.
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.270, b: -0.312 Cost: 34.129803
Epoch  100/1000 W: 1.801, b: 0.452 Cost: 0.029475
Epoch  200/1000 W: 1.844, b: 0.355 Cost: 0.018214
Epoch  300/1000 W: 1.877, b: 0.279 Cost: 0.011255
Epoch  400/1000 W: 1.903, b: 0.220 Cost: 0.006955
Epoch  500/1000 W: 1.924, b: 0.173 Cost: 0.004298
Epoch  600/1000 W: 1.940, b: 0.136 Cost: 0.002656
Epoch  700/1000 W: 1.953, b: 0.107 Cost: 0.001641
Epoch  800/1000 W: 1.963, b: 0.084 Cost: 0.001014
Epoch  900/1000 W: 1.971, b: 0.066 Cost: 0.000627
Epoch 1000/1000 W: 1.977, b: 0.052 Cost: 0.000387
